In [147]:
import tweepy  # https://github.com/tweepy/tweepy
import csv
import json
import sys
import datetime
from pytz import timezone
import hashlib
import pandas as pd

In [4]:
# CREDENTIALS
api_key = "zGWacA8CkRVNQgY5vgAh4nfAp"
api_secret_key = "DWaRZpAls3u4YcotLKcImKeYy03qlcBqXBKlDr6CtK5Y33Jc3I"
access_token = "1140211942436429825-5D5hkMjOGr5WyxciM8qO7WH6lz0cOq"
access_token_secret = "AZZ0OhhV2t8W1n9Q8MqopgtvZaChXsZzxwr1CbF9OmcIz"

auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [128]:
# Total 33 companied of DJIA
companies = "3M,Walgreens,Salesforce,Amgen,Honeywell,American Express,Apple,Boeing,Caterpillar,Chevron,Cisco Systems," \
            "Coca-Cola,ExxonMobil,General Electric,Goldman Sachs,IBM," \
            "Intel,Johnson & Johnson,JPMorgan Chase,McDonald's,Merck,Microsoft,Nike," \
            "Pfizer,Procter & Gamble,The Home Depot,Travelers,United Technologies," \
            "UnitedHealth Group,Verizon,Visa,Walmart,Walt Disney".split(",")

accounts = "3M,Walgreens,Salesforce,Amgen,Honeywell,AmericanExpress,AppleSupport,Boeing,CaterpillarInc,Chevron,Cisco," \
           "CocaCola,exxonmobil,generalelectric,GoldmanSachs,IBM,intel," \
           "JNJNews,jpmorgan,McDonalds,Merck,Microsoft,Nike,pfizer,ProcterGamble," \
           "HomeDepot,Travelers,UTC,UnitedHealthGrp,verizon,Visa,Walmart,DisneyStudios".split(",")

comDic = dict(zip(accounts, companies))

In [129]:
comDic

{'3M': '3M',
 'Walgreens': 'Walgreens',
 'Salesforce': 'Salesforce',
 'Amgen': 'Amgen',
 'Honeywell': 'Honeywell',
 'AmericanExpress': 'American Express',
 'AppleSupport': 'Apple',
 'Boeing': 'Boeing',
 'CaterpillarInc': 'Caterpillar',
 'Chevron': 'Chevron',
 'Cisco': 'Cisco Systems',
 'CocaCola': 'Coca-Cola',
 'exxonmobil': 'ExxonMobil',
 'generalelectric': 'General Electric',
 'GoldmanSachs': 'Goldman Sachs',
 'IBM': 'IBM',
 'intel': 'Intel',
 'JNJNews': 'Johnson & Johnson',
 'jpmorgan': 'JPMorgan Chase',
 'McDonalds': "McDonald's",
 'Merck': 'Merck',
 'Microsoft': 'Microsoft',
 'Nike': 'Nike',
 'pfizer': 'Pfizer',
 'ProcterGamble': 'Procter & Gamble',
 'HomeDepot': 'The Home Depot',
 'Travelers': 'Travelers',
 'UTC': 'United Technologies',
 'UnitedHealthGrp': 'UnitedHealth Group',
 'verizon': 'Verizon',
 'Visa': 'Visa',
 'Walmart': 'Walmart',
 'DisneyStudios': 'Walt Disney'}

In [130]:
# function to get the tweets base on their screen_name(user_id)
def get_all_tweets(screen_name):
    # Twitter only allows access to a users most recent 3200 tweets with this method

    print("collecting tweets from: " + screen_name)
    # initialize a list to hold all the tweepy Tweets
    alltweets = []

    # make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name=screen_name, count=200)

    # save most recent tweets
    alltweets.extend(new_tweets)

    # save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    # keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        # all subsequent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name=screen_name, count=200,
                                       max_id=oldest, exclude_replies=True)

        # save most recent tweets
        alltweets.extend(new_tweets)

        # update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))

    # transform the tweepy tweets into a 2D array that will populate the csv
    outtweets = [[
        tweet.created_at,
        comDic[screen_name],
        "https://twitter.com/" + screen_name + "/status/" + tweet.id_str,
        tweet.text.encode("utf-8"),
        tweet.retweet_count,
        tweet.favorite_count,
        tweet.id_str]
        for tweet in alltweets]

    # compose dataframe for outputting
    df = pd.DataFrame(columns=["created_at", "company", "url", "tweets", "re_tweets", "likes", "id"])
    for tweet in outtweets:
        row = pd.Series(tweet,index=["created_at", "company", "url", "tweets", "re_tweets", "likes", "id"])
        df = df.append(row, ignore_index=True)
        
    return df

In [131]:
allData_list = []
for acc in accounts:
    allData_list.append(get_all_tweets(acc))

collecting tweets from: 3M
...254 tweets downloaded so far
...294 tweets downloaded so far
...355 tweets downloaded so far
...408 tweets downloaded so far
...467 tweets downloaded so far
...534 tweets downloaded so far
...576 tweets downloaded so far
...622 tweets downloaded so far
...678 tweets downloaded so far
...727 tweets downloaded so far
...749 tweets downloaded so far
...766 tweets downloaded so far
...799 tweets downloaded so far
...831 tweets downloaded so far
...863 tweets downloaded so far
...893 tweets downloaded so far
...893 tweets downloaded so far
collecting tweets from: Walgreens
...206 tweets downloaded so far
...207 tweets downloaded so far
...209 tweets downloaded so far
...210 tweets downloaded so far
...211 tweets downloaded so far
...212 tweets downloaded so far
...214 tweets downloaded so far
...225 tweets downloaded so far
...229 tweets downloaded so far
...236 tweets downloaded so far
...246 tweets downloaded so far
...250 tweets downloaded so far
...257 twee

...2310 tweets downloaded so far
...2510 tweets downloaded so far
...2710 tweets downloaded so far
...2910 tweets downloaded so far
...3110 tweets downloaded so far
...3160 tweets downloaded so far
...3160 tweets downloaded so far
collecting tweets from: IBM
...217 tweets downloaded so far
...231 tweets downloaded so far
...254 tweets downloaded so far
...278 tweets downloaded so far
...290 tweets downloaded so far
...295 tweets downloaded so far
...304 tweets downloaded so far
...319 tweets downloaded so far
...328 tweets downloaded so far
...336 tweets downloaded so far
...347 tweets downloaded so far
...358 tweets downloaded so far
...371 tweets downloaded so far
...381 tweets downloaded so far
...396 tweets downloaded so far
...407 tweets downloaded so far
...407 tweets downloaded so far
collecting tweets from: intel
...338 tweets downloaded so far
...440 tweets downloaded so far
...522 tweets downloaded so far
...572 tweets downloaded so far
...636 tweets downloaded so far
...698 

...729 tweets downloaded so far
...745 tweets downloaded so far
...853 tweets downloaded so far
...948 tweets downloaded so far
...1037 tweets downloaded so far
...1116 tweets downloaded so far
...1191 tweets downloaded so far
...1232 tweets downloaded so far
...1232 tweets downloaded so far
collecting tweets from: Visa
...235 tweets downloaded so far
...253 tweets downloaded so far
...257 tweets downloaded so far
...258 tweets downloaded so far
...259 tweets downloaded so far
...261 tweets downloaded so far
...262 tweets downloaded so far
...263 tweets downloaded so far
...330 tweets downloaded so far
...330 tweets downloaded so far
collecting tweets from: Walmart
...226 tweets downloaded so far
...227 tweets downloaded so far
...229 tweets downloaded so far
...231 tweets downloaded so far
...232 tweets downloaded so far
...234 tweets downloaded so far
...239 tweets downloaded so far
...240 tweets downloaded so far
...243 tweets downloaded so far
...249 tweets downloaded so far
...258

In [137]:
allData_df['tweets'][0]

b'@doddthesod Hello Simon - We are sorry to see this. It is not what we would expect. Please connect with our UK supp\xe2\x80\xa6 https://t.co/syixQ1BNQT'

In [133]:
allData_df = pd.concat(allData_list).reset_index(drop=True)
allData_df

,created_at,company,url,tweets,re_tweets,likes,id
0,2022-08-12 15:13:06+00:00,3M,https://twitter.com/3M/status/1558109327424163840,b'@doddthesod Hello Simon - We are sorry to se...,0,0,1558109327424163840
1,2022-08-11 20:49:19+00:00,3M,https://twitter.com/3M/status/1557831550422728705,b'@bleepingED Hello - Thank you for reaching o...,0,1,1557831550422728705
2,2022-08-11 18:09:52+00:00,3M,https://twitter.com/3M/status/1557791422627123200,b'Did you know stop signs have not always been...,2,11,1557791422627123200
3,2022-08-11 16:32:22+00:00,3M,https://twitter.com/3M/status/1557766888775720960,b'No nails? No screws? No problem!\n\n@command...,2,11,1557766888775720960
4,2022-08-11 15:03:20+00:00,3M,https://twitter.com/3M/status/1557744482048770048,b'@crazypnut Hello - Thank you for reaching ou...,0,0,1557744482048770048
...,...,...,...,...,...,...,...
55190,2019-04-07 19:06:00+00:00,Walt Disney,https://twitter.com/DisneyStudios/status/11149...,"b'In Dreamland, everything is possible. See #D...",104,937,1114967577883762691
55191,2019-04-07 16:00:00+00:00,Walt Disney,https://twitter.com/DisneyStudios/status/11149...,b'Celebrate 10 years of @Disneynature in advan...,100,734,1114920771753844736
55192,2019-04-06 19:00:00+00:00,Walt Disney,https://twitter.com/DisneyStudios/status/11146...,"b'See the #1 movie in the world, in theaters n...",51,522,1114603682195095553
55193,2019-04-06 18:00:01+00:00,Walt Disney,https://twitter.com/DisneyStudios/status/11145...,b'There\xe2\x80\x99s no one quite like Steve. ...,207,1340,1114588585485918208


In [134]:
allData_df.describe()

C:\Users\whwen\AppData\Local\Temp/ipykernel_12952/3633397753.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  allData_df.describe()


,created_at,company,url,tweets,re_tweets,likes,id
count,55195,55195,55195,55195,55195.0,55195.0,55195
unique,54753,33,55195,55058,1026.0,2033.0,55195
top,2022-08-08 14:18:14+00:00,Goldman Sachs,https://twitter.com/3M/status/1558109327424163840,b'Here are 5 ways Amgen supports the environme...,0.0,0.0,1558109327424163840
freq,4,3160,1,10,5612.0,12192.0,1
first,2012-05-17 12:48:23+00:00,NaN,NaN,NaN,NaN,NaN,NaN
last,2022-08-13 13:15:04+00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
allData_df.to_pickle("Merge_data.pkl")

In [156]:
start_date = "2022-06-01"
start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
end_date = "2022-07-01"
end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")

# Right way!
start_date = start_date.replace(tzinfo=timezone('UTC'))
end_date = end_date.replace(tzinfo=timezone('UTC'))

In [157]:
print(start_date)
print(end_date)

2022-06-01 00:00:00+00:00
2022-07-01 00:00:00+00:00


In [159]:
allData_df.query(' created_at >= @start_date and created_at <= @end_date').reset_index(drop=True)

,created_at,company,url,tweets,re_tweets,likes,id
0,2022-06-30 18:50:06+00:00,3M,https://twitter.com/3M/status/1542581260689833986,b'Happy #SocialMediaDay \xf0\x9f\x93\xb1 Join ...,2,16,1542581260689833986
1,2022-06-30 17:43:50+00:00,3M,https://twitter.com/3M/status/1542564580563582982,"b""@sevenpiggies Hello - Thank you for messagin...",0,0,1542564580563582982
2,2022-06-29 17:28:44+00:00,3M,https://twitter.com/3M/status/1542198395275706369,b'@ohunt Sorry to hear of the difficulty! Plea...,0,0,1542198395275706369
3,2022-06-29 17:01:30+00:00,3M,https://twitter.com/3M/status/1542191540227002370,b'See how artist @gabe_gault uses artificial r...,0,15,1542191540227002370
4,2022-06-29 16:56:50+00:00,3M,https://twitter.com/3M/status/1542190367130402821,b'@mochesa Hello! The 3M Kenya help center can...,0,1,1542190367130402821
...,...,...,...,...,...,...,...
1790,2022-06-03 16:00:04+00:00,Walt Disney,https://twitter.com/DisneyStudios/status/15327...,b'\xf0\x9f\x8e\xb6 ...all the flowers and vine...,29,153,1532753996456374274
1791,2022-06-03 15:00:03+00:00,Walt Disney,https://twitter.com/DisneyStudios/status/15327...,"b""Here's to the dreamers! \xf0\x9f\x92\xab\xf0...",28,187,1532738892004139008
1792,2022-06-02 19:30:41+00:00,Walt Disney,https://twitter.com/DisneyStudios/status/15324...,"b""LA state of mind. \xe2\x98\x80\xef\xb8\x8f\x...",20,168,1532444613109174272
1793,2022-06-02 17:00:02+00:00,Walt Disney,https://twitter.com/DisneyStudios/status/15324...,b'We all have our own stories to tell \xf0\x9f...,18,162,1532406699599925266
